# Template Notebook

### Configuration

In [15]:
import os
import sys
import subprocess
import zipfile
import json

In [2]:
# set globals
USER = 'josemarquezjaramillo'
REPO = 'crypto-rl-portfolio'
DATASET = 'dataset_v1'

In [3]:
# pull repo - colab starts uninitialized
os.system(f'git clone https://github.com/{USER}/{REPO}')
os.chdir(REPO)

# install requirements
req_result = subprocess.run('pip install -r requirements-data.txt', shell=True, capture_output=True, text=True)

print("=== STDOUT ===")
print(req_result.stdout.strip())   # remove extra blank lines
print("\n=== STDERR ===")
print(req_result.stderr.strip())

print(f"\nExit code: {req_result.returncode}")

=== STDOUT ===
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 120.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.6/250.6 kB 26.5 MB/s eta 0:00:00

=== STDERR ===


Exit code: 0


In [41]:
# extract data
with zipfile.ZipFile(f"data/{DATASET}.zip", "r") as zip_ref:
    zip_ref.extractall(".")

#### Imports

In [16]:
sys.path.append(os.getcwd())

In [17]:
# library imports
import pandas as pd
import numpy as np

# local imports
from agents import *
from environment import *
from tests import *

# pytorch
import torch
# get cuda
print(torch.cuda.is_available())

True


#### Read Data

In [42]:
def read_asset_lists(subset):
    data = []
    with open(f"{DATASET}/{subset}_asset_lists.jsonl") as f:
        for line in f:
            json_object = json.loads(line.strip())
            data.append(json_object)
    return data

def read_index(subset):
  return pd.read_parquet(f"{DATASET}/{subset}_index.parquet")

def read_fwd_returns(subset):
  return np.load(f"{DATASET}/{subset}_fwd_returns.npz")

def read_obs_tensors(subset):
  return np.load(f"{DATASET}/{subset}_obs_tensors.npz")

metadata = json.load(open(f"{DATASET}/metadata.json"))
dev_asset_list = read_asset_lists('dev')
dev_index = read_index('dev')
dev_tensors = read_obs_tensors('dev')

In [43]:
def get_obs(idx_tbl, tensors, idx):
  date = idx_tbl['date'][idx].strftime('t_%Y-%m-%d')
  return tensors[date]

In [44]:
data = get_obs(dev_index, dev_tensors, 0)

In [45]:
data.shape

(10, 4, 60)

## Run Example Code (Environment)

In [48]:
!python -m environment.environment_smoke_run

SMOKE TEST: PortfolioEnv End-to-End Demonstration

[1/6] Loading dataset artifacts...
  ✓ Loaded 960 days from dataset_v1/dev

[2/6] Creating data backends...
  ✓ Training backend: 922 days
  ✓ Validation backend: 38 days (2 windows)

[3/6] Initializing environments...
  ✓ Training env: 65 days
  ✓ Validation env: 19 days

[4/6] Initializing agents...
  ✓ Random agent (exponential weights)
  ✓ Uniform agent (1/N baseline)

[5/6] Training agents...

  Random Agent (Training):
  Episode 1/3... PV=0.8887, reward=-0.1180
  Episode 2/3... PV=1.0048, reward=0.0048
  Episode 3/3... PV=0.8382, reward=-0.1765

  Uniform Agent (Training):
  Episode 1/3... PV=0.8922, reward=-0.1141
  Episode 2/3... PV=0.8922, reward=-0.1141
  Episode 3/3... PV=0.8922, reward=-0.1141

[6/6] Evaluating on validation set...

  Random Agent (Validation):
  Episode 1/1... PV=0.6873, reward=-0.3749

  Uniform Agent (Validation):
  Episode 1/1... PV=0.6729, reward=-0.3961

RESULTS SUMMARY

Training Performance (5 months